# Plot Individual Traces
**Description:** This notebook visualizes the signals for a single embryo.  

* Dff trace
* Partial dff trace
* Spectrogram

## Import Libraries

In [3]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import matplotlib as mpl

from pasna_analysis import Experiment, myplots

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load Data

Specify the location of the data by editing `folder` and `experiment_name`. Then specify the embryo.

In [4]:
folder = 'rdl' # HERE
experiment_name = '20250518_RdlMutant_R' # HERE

to_exclude = []

exp_path = Path.cwd().parent.joinpath('data', folder, experiment_name)
exp = Experiment(exp_path, first_peak_threshold=0, 
        to_exclude=to_exclude, dff_strategy='local_minima', hatches=True)

embryos = list(exp.embryos.values())
emb = embryos[0] # HERE
trace = emb.trace

print(f'Loading {folder} {experiment_name} {emb.name}')

ValidationError: 1 validation error for ConfigObj
exp_path
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing

## Configure Plot Style

Customize the Matplotlib plots with rcParams. For a full list of parameters see [matplotlib.rcParams](https://matplotlib.org/stable/api/matplotlib_configuration_api.html#default-values-and-styling)

In [ ]:
myplots.set_darkbg(False)

trace_rc = {
    'figure.figsize': (35, 5),
    'figure.frameon': False,

    'lines.linewidth': 3,

    'font.family': 'Arial',
    'font.size': 40,

    'xtick.major.size': 10, # length
    'xtick.major.width': 2,
    'ytick.major.size': 10, # length
    'ytick.major.width': 2,

    'axes.spines.top': False,
    'axes.spines.right': False,
    'axes.labelpad': 10,
    'axes.linewidth': 2.5
}

partial_trace_rc = {
    'figure.figsize': (20, 8),
    'figure.frameon': False,

    'lines.linewidth': 4,

    'font.family': 'Arial',
    'font.size': 40,

    'xtick.major.size': 10, # length
    'xtick.major.width': 2,
    'ytick.major.size': 10, # length
    'ytick.major.width': 2,

    'axes.spines.top': False,
    'axes.spines.right': False,
    'axes.labelpad': 10,
    'axes.linewidth': 2.5
}

spec_rc = {
    'figure.figsize': (35, 8),
    'figure.frameon': False,

    'font.family': 'Arial',
    'font.size': 40,

    'axes.spines.top': False,
    'axes.spines.right': False,

    'xtick.major.size': 10, # length
    'xtick.major.width': 2,
    'ytick.major.size': 10, # length
    'ytick.major.width': 2,
    'axes.labelpad': 10,
    'axes.linewidth': 2.5
}

mymap = mpl.colormaps['turbo']
mymap.set_over('black')

## Plot DFF

Preprocesses the dff to a standardized length, then plots it over real time.

**preprocess_dff**

optional parameters:
* duration: target number of frames to include
* onset_pad: number of frames to include before onset


**plot_trace** 

optional parameters:
* color: line color
* xmin: x-axis minimum in minutes
* xmax: x-axis maximum in minutes
* xinterval: num of minutes between ticks
* ymin: y-axis minimum in minutes
* ymax: y-axis maximum in minutes

note: if you dont specify a color, one will be chosen for you

In [ ]:
time, dff = trace.preprocess_dff()
myplots.plot_trace(time, dff, trace_rc, ymax=2)

## Plot Partial DFF

Preprocesses the dff to a standardized length, then plots it over real time.

**preprocess_dff** 

optional parameters:
* duration: target number of frames to include
* onset_pad: number of frames to include before onset


**plot_trace** 

optional parameters:
* color: line color
* xmin: x-axis minimum in minutes
* xmax: x-axis maximum in minutes
* xinterval: num of minutes between ticks
* ymin: y-axis minimum in minutes
* ymax: y-axis maximum in minutes

note: if you dont specify a color, one will be chosen for you

In [ ]:
time, dff = trace.preprocess_dff()
myplots.plot_trace(time, dff, partial_trace_rc, xmin=165, xmax=195, xinterval=5, ymax=1)

## Plot Spectrogram

Preprocesses the dff to a standardized length, then plots the magnitude of its STFT over time. The x-axis displays time (columns), y-axis displays frequency (rows), and color displays intensity. 

**preprocess_dff** 

optional parameters:
* duration: target number of frames to include
* onset_pad: number of frames to include before onset


**calculate_STFT**

optional parameters:
* fs: sampling rate
* fft_size: num frames in each segment
* noverlap: num frames to overlap between segements

**plot_spec** 

optional parameters:
* display_colorbar: True to display, False to hide
* xmin: x-axis minimum in minutes
* xmax: x-axis maximum in minutes
* ymin: y-axis minimum in minutes
* ymax: y-axis maximum in minutes

In [ ]:
time, dff = trace.preprocess_dff()
f, t, Zxx = trace.calculate_STFT(dff)
myplots.plot_spec(f, t, Zxx, mymap, spec_rc, display_colorbar=False)